In [1]:
import sys
sys.path.append('../ML_Analysis')

In [2]:
import pandas as pd
import numpy as np
import gensim as gn
import tweepy
from parse import parse

import pickle

In [13]:
from AdaptiveOnlineClustering import AdaptiveOnlineClustering as aoc

In [5]:
path = 'Downloads/sehir_matches.csv'
sehir_twitter = pd.read_csv(path)

In [ ]:
template = 'api_key:{api_key};api_secret:{api_secret};access_token:{access_token};access_secret:{access_secret};'

with open('cr', 'r') as f:
    credentials = parse(template, f.read().replace('\n', ''))

In [ ]:
auth = tweepy.OAuthHandler(credentials['api_key'], credentials['api_secret'])
auth.set_access_token(credentials['access_token'], credentials['access_secret'])

api = tweepy.API(auth)

In [ ]:
sample = np.random.choice(sehir_twitter.twitter_screen_name, size=1000, replace=False)
statuses = []

for user in sample:
    try:
        statuses.append(api.user_timeline(user))
        
    except Exception as e:
        print(e, user)

In [ ]:
len(statuses)

In [ ]:
pickle.dump(statuses, open('tweets.pkl', 'wb'))

In [12]:
model = gn.models.KeyedVectors.load_word2vec_format('/home/ammar/NLP_data/GoogleNews-vectors-negative300.bin', binary=True)  

In [22]:
tweets = pickle.load(open('../datasets/tweets.pkl', 'rb'))

In [6]:
from sklearn.cluster import MiniBatchKMeans, Birch

In [15]:
a = aoc(model, None)

In [39]:
all_tweets = []

for user in tweets:
    for tweet in user:
        if tweet.lang == 'en':
            all_tweets.append(tweet)

In [40]:
len(all_tweets)

878

In [47]:
tweets_v = [a.vectorize(i.text, language='en') for i in all_tweets]

In [48]:
np.array(tweets_v).shape

(878, 300)

In [54]:
mbk = MiniBatchKMeans(n_clusters=500).fit(tweets_v)

/usr/local/lib/python3.5/dist-packages/sklearn/cluster/k_means_.py:1418: RuntimeWarning: init_size=300 should be larger than k=500. Setting it to 3*k
  init_size=init_size)


In [57]:
mbk.counts_

array([  7,   8,   3,   5, 112,  11,   2,   5,   2,   4, 119,   6,   4,
        26,   4,   4,   1,  25,  11,   6,   3,   3,   2,   1,   1,   2,
         3,   4,   7,  25,   9,   2,   4,  13,  20,   6,   4,   6,   1,
         6,   2,   3,   2,   2,   2,   3,   4, 105,   2,   9,   3,   3,
        13,   5,   2,   2,   1,   3,   4,   3,   3,   3,   4,   1,   5,
         2,   5,   1,   4,   3,   1,   3,   3,  30,   3,   3,   2,  77,
         3,   1,   1,   1,   3,   2,   5,   2,   2,   3,   2,   1,   2,
         5,   4,   2,  17,   1,   5,   8,  10,   2,   2,   1,   6,   1,
         1,  18,   2,   4,   1,   1,   3,   2,   2,   1,   2,   2,   5,
         3,   4,   1,   1,   9,   5,   2,   7,   4,   2,   1,   3,   3,
         1,   4,   6,   1,   3,   2,   3,   1,   1,   1,   1,   6,   1,
         3,   2,   2,   5,   1,   2,   1,   2,   3,   2,   4,   1,   2,
         3,   6,   4,   3,   1,   1,   1,   1,   2,   1,   4,   3,   2,
         4,   3,   2,   2,   4,   8,   1,  10,   1,   2,   3,   

In [10]:
for t in tweets[0][:100]:
    lang = t.lang
    if lang != "en": continue
    text = t.text
    vect = a.vectorize(text, lang)
#     mbk.partial_fit(vect)
    mbk.fit_transform(vect)
    

In [9]:
for t in tweets[1][:100]:
    lang = t.lang
    if lang != "en": continue
    text = t.text
    vect = a.vectorize(text, lang)
    mbk.partial_fit(vect)
    

In [10]:
mbk.c

AttributeError: 'MiniBatchKMeans' object has no attribute 'count_'